# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Import and clean data
weatherData = pd.read_csv('../WeatherPy/Output/weatherData.csv')
weatherData.dropna(inplace=True)
weatherData.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,ginir,7.13,40.70,53.64,61,0,4.09,ET,1604364167
1,biak,-0.91,122.88,82.60,56,100,2.21,ID,1604363827
2,wewak,-3.55,143.63,86.74,64,95,1.45,PG,1604364167
3,puerto madero,14.72,-92.42,82.40,88,75,2.24,MX,1604364167
4,ust-koksa,50.27,85.61,28.49,86,100,1.59,RU,1604364168


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Use the lat/lng as locations and humidity as the weight
locations = weatherData[['Lat', 'Lng']].astype(float)

# Add heatmap layer to map
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations,
    weights=weatherData['Humidity'],
    dissipating=False,
    max_intensity=100,
    point_radius=1
)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Pull only cities fitting my ideal conditions
myDreamDestinations_df = weatherData.loc[(weatherData['Cloudiness'] < 10) &  (weatherData['Max Temp'] >= 70) & (weatherData['Max Temp'] <= 90) & (weatherData['Wind Speed'] < 10)]
myDreamDestinations_df.dropna(inplace=True)
myDreamDestinations_df.head()

<ipython-input-23-55a64fd81588>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  myDreamDestinations_df.dropna(inplace=True)


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
27,kavaratti,10.57,72.64,82.04,69,0,4.61,IN,1604363790
34,lazaro cardenas,17.96,-102.20,81.84,74,0,9.13,MX,1604363836
48,tabou,4.42,-7.35,77.92,86,0,4.16,CI,1604364178
50,jizan,17.33,42.67,82.40,74,0,5.82,SA,1604364127
54,karratha,-20.74,116.85,83.70,26,0,4.12,AU,1604364180


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [33]:
# Store destinations in DataFrame
hotel_df = myDreamDestinations_df.copy()
# Add Hotel column
hotel_df['Hotel Name'] = ''
hotel_df.head()

# Set paramaters to search for hotels within 5000 meters
params = {
    'radius': 5000,
    'types': 'lodging',
    'keyword': 'Hotel',
    'key': g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Hit up the Google Places API for each city's coordinates
for index, row in hotel_df.iterrows():
    # Add location parameter for each city
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    
    # Make request
    response = requests.get(base_url, params=params).json()
    # print(response)
    
    # Try/Except in case data is missing
    try:
        # Store data in data frame
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
        
    except:
        print(f"Missing result, skipping {index}")
        
hotel_df.head()

Missing result, skipping 50
Missing result, skipping 117
Missing result, skipping 310
Missing result, skipping 314
Missing result, skipping 356


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
27,kavaratti,10.57,72.64,82.04,69,0,4.61,IN,1604363790,yaad Hotal
34,lazaro cardenas,17.96,-102.20,81.84,74,0,9.13,MX,1604363836,Hotel de Casablanca
48,tabou,4.42,-7.35,77.92,86,0,4.16,CI,1604364178,hotêl doufoulougou
50,jizan,17.33,42.67,82.40,74,0,5.82,SA,1604364127,
54,karratha,-20.74,116.85,83.70,26,0,4.12,AU,1604364180,Latitude20 The Dunes Apartments


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

AttributeError: 'list' object has no attribute 'iterrows'

In [38]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(
    locations,
    info_box_content = hotel_df
)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))